In [1]:
import os.path as op
import matplotlib.pyplot as plt
from itertools import compress
import time
import logging
import mne
from mne import events_from_annotations
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from itertools import compress
import mne
import mne_nirs
from mne.preprocessing.nirs import optical_density, beer_lambert_law

from mne_nirs.signal_enhancement import enhance_negative_correlation, short_channel_regression

from mne.preprocessing.nirs import (optical_density,
                                    temporal_derivative_distribution_repair,
                                    scalp_coupling_index)

import ROI
from ROI import different_hb, different_roi
from meta import *
from filter_params import FILTER_DICT
from functions_fnirs import *

def fast_scanfiles(dirname, contains=None):
    subfiles = [f.path for f in os.scandir(dirname) if f.is_file()]
    if contains != None:
        subfiles = [i for i in subfiles if contains in i ]
    return subfiles

C:\Users\Admin\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [71]:
from matplotlib.colors import LinearSegmentedColormap

fnirs_colors = dict(
    hbo='#C91111',
    hbt= '#5B324B',
    hbr='#004E7C',
  )

legend_params_dict = dict(
                            loc='center', 
                            fontsize=11,       
                            borderpad=1.5, 
                            labelspacing=1.5,
                            markerscale=10, 
                            framealpha=0
)

curves_subplot_params = dict(
 top=0.92,
 bottom=0.11,
 left=0.065,
 right=0.945,
 hspace=0.2,
 wspace=0.35
)

two_topomaps_subplot_params = dict(top=0.89,
bottom=0.21,
left=0.115,
right=0.98,
hspace=0.175,
wspace=0.0)

one_topomap_subplot_params = dict(top=0.92,
bottom=0.075,
left=0.105,
right=0.98,
hspace=0.105,
wspace=0.0)


mask_params = dict(marker='o', 
                   markerfacecolor='black', 
                   markeredgecolor='white',
                   linewidth=0, 
                   markersize=6
                   )

color_list = [(0, fnirs_colors['hbr']), (0.5, '#FFFFE4'), (1, fnirs_colors['hbo'])]
custom_cmap = LinearSegmentedColormap.from_list('Custom_cmap', color_list, )

In [52]:
haemo_dict = dict(
    
ME_smr_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='ME_smr_left')],
MI_smr_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='MI_smr_left')],
TS_smr_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TS_smr_left')],
TI_smr_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TI_smr_left')],
SA_smr_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='SA_smr_left')],

ME_rest_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='ME_rest_left')],
MI_rest_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='MI_rest_left')],
TS_rest_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TS_rest_left')],
TI_rest_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TI_rest_left')],
SA_rest_left = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='SA_rest_left')],


ME_smr_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='ME_smr_right')],
MI_smr_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='MI_smr_right')],
TS_smr_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TS_smr_right')],
TI_smr_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TI_smr_right')],
SA_smr_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='SA_smr_right')],

ME_rest_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='ME_rest_right')],
MI_rest_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='MI_rest_right')],
TS_rest_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TS_rest_right')],
TI_rest_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='TI_rest_right')],
SA_rest_right = [np.load(i) for i in fast_scanfiles(r"C:\Users\Admin\Desktop\fnirs_python_code\haemodynamics_np",
                                        contains='SA_rest_right')],
)



ME_subdict = make_subdicts(haemo_dict, contains='ME')
MI_subdict = make_subdicts(haemo_dict, contains='MI')
TS_subdict = make_subdicts(haemo_dict, contains='TS')
TI_subdict = make_subdicts(haemo_dict, contains='TI')
SA_subdict = make_subdicts(haemo_dict, contains='SA')

In [58]:
def make_subdicts(haemo_dict, contains):
    new_dict = {key[3:]:value for key, value in haemo_dict.items() if contains in key}
    return new_dict


def concatenate_dicts(haemo_dict):
    new_dict = {}
    for key in haemo_dict.keys():
        a = np.vstack(haemo_dict[key])
        a = a.reshape(20, 32, 30)
        new_dict[key] = a
    return new_dict

def get_roi_evokeds(new_dict, key):
    M1 = new_dict[key].mean(axis=0)[:6, :].mean(axis=0)
    S1 = new_dict[key].mean(axis=0)[6:13, :].mean(axis=0)
    SMZ = new_dict[key].mean(axis=0)[13:, :].mean(axis=0)
    return M1, S1, SMZ

ME_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='ME'))
MI_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='MI'))
TS_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='TS'))
TI_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='TI'))
SA_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='SA'))

In [94]:
subdict = TI_subdict

M1_evoked_SMR_left, S1_evoked_SMR_left, SMZ_evoked_SMR_left = get_roi_evokeds(subdict, key='smr_left')
M1_evoked_REST_left, S1_evoked_REST_left, SMZ_evoked_REST_left = get_roi_evokeds(subdict, key='rest_left')
M1_evoked_SMR_right, S1_evoked_SMR_right, SMZ_evoked_SMR_right = get_roi_evokeds(subdict, key='smr_right')
M1_evoked_REST_right, S1_evoked_REST_right, SMZ_evoked_REST_right = get_roi_evokeds(subdict, key='rest_right')


In [95]:
%matplotlib auto
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

Using matplotlib backend: QtAgg


In [96]:
info_hbr_total = mne.io.read_info(r"C:\Users\Admin\Desktop\fnirs_python_code\info_hbr_total_info.fif")
info_left_smz = mne.io.read_info(r"C:\Users\Admin\Desktop\fnirs_python_code\info_left_smz_info.fif")
info_right_smz = mne.io.read_info(r"C:\Users\Admin\Desktop\fnirs_python_code\info_right_smz_info.fif")
info_hbo_total = mne.io.read_info(r"C:\Users\Admin\Desktop\fnirs_python_code\info_hbo_total_info.fif")

In [97]:
### PLOT HEMODYNAMICS PER SE ###
m1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['M1'][0] ]
s1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['S1'][0] ]
smz_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][0]]

m1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['M1'][2] ]
s1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['S1'][2] ]
smz_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][2]]


curves_hb = 'hbo'
SUBJECT = 'AVERAGE'
CONDITION = 'ME'

times = np.arange(TMIN, TMAX, 1/2)
linewidth = 1.5
ylims = (-1000, 1500)
tmin, tmax = TMIN, TMAX
topo_linewidth = 1
pointsize = 20
topo_width = topo_height = '30%'

fig, axes = plt.subplots(1, 2, figsize=(20, 12))

ylims=(-12, 16)


### LEFT PART ###

line_m1_smr, = axes[0].plot(times, M1_evoked_SMR_left*1e6, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
line_s1_smr, = axes[0].plot(times, S1_evoked_SMR_left*1e6, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
line_smz_smr, = axes[0].plot(times, SMZ_evoked_SMR_left*1e6, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])
line_m1_rest, = axes[0].plot(times, M1_evoked_REST_left*1e6, label=f'M1/{curves_hb} REST', 
         color=fnirs_colors['hbr'], linestyle='--')
line_s1_rest, = axes[0].plot(times, S1_evoked_REST_left*1e6, label=f'S1/{curves_hb} REST', 
         color=fnirs_colors['hbo'], linestyle='--')
line_smz_rest, = axes[0].plot(times, SMZ_evoked_REST_left*1e6, label=f'SMR/{curves_hb} REST', 
         color=fnirs_colors['hbt'], linestyle='--')

fill_1 = filler_between(axes[0], ylims)

set_axis_properties(axes[0],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO response in LEFT hemisphere\nSubject {SUBJECT} Condition {CONDITION}',
                    xlabel='Time, s', 
                    ylabel='Hb concentration, Δ μM\L',  
                    linewidth=1.5, 
                    fontsize=14, 
                    title_size=18,
                    legend_flag=False
                        )
### LEFT HEMO TOPOMAP ###
inset_ax1 = inset_axes(axes[0], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(info_left_smz, 
                     ch_groups=[  m1_group_left, 
                                 smz_group_left,
                                s1_group_left],
                     axes=inset_ax1,
                     pointsize=pointsize, 
                     linewidth=topo_linewidth)

### RIGHT PART ###
axes[1].plot(times, M1_evoked_SMR_right*1e6, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
axes[1].plot(times, S1_evoked_SMR_right*1e6, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
axes[1].plot(times, SMZ_evoked_SMR_right*1e6, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])
axes[1].plot(times, M1_evoked_REST_right*1e6, label=f'M1/{curves_hb} REST', 
         color=fnirs_colors['hbr'], linestyle='--')
axes[1].plot(times, S1_evoked_REST_right*1e6, label=f'S1/{curves_hb} REST', 
         color=fnirs_colors['hbo'], linestyle='--')
axes[1].plot(times, SMZ_evoked_REST_right*1e6, label=f'SMR/{curves_hb} REST', 
         color=fnirs_colors['hbt'], linestyle='--')

fill_2 = filler_between(axes[1], ylims)


set_axis_properties(axes[1],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO response in RIGHT hemisphere\nSubject {SUBJECT} Condition {CONDITION}',
                    xlabel='Time, s', 
                    ylabel='',  
                    linewidth=1.5, 
                    fontsize=14, 
                    title_size=18,
                    legend_flag=False,
                        )

### RIGHT HEMO TOPOMAP ###
inset_ax2 = inset_axes(axes[1], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(
                    info_right_smz, 
                    ch_groups=[
                                m1_group_right, 
                                smz_group_right,
                                s1_group_right],
                    axes=inset_ax2, 
                    pointsize=pointsize, 
                    linewidth=topo_linewidth)
                     

### LEGENDS PARAMS ###
ax_legend = fig.add_axes([0.25, 0.25, 0.5, 0.25])
ax_legend.legend([line_m1_smr, line_s1_smr, line_smz_smr,
                 line_m1_rest, line_s1_rest, line_smz_rest, fill_1], 
                 [f'M1/{curves_hb} SMR', f'S1/{curves_hb} SMR', f'SMZ/{curves_hb} SMR', 
                  f'M1/{curves_hb} REST', f'S1/{curves_hb} REST',f'SMZ/{curves_hb} REST',
                 'Task duration'], 
                **legend_params_dict) 

ax_legend.set_frame_on(False)
ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

plt.subplots_adjust(**curves_subplot_params)


